<div class='alert alert-block alert-info'>Ophalen belasting-waarden vanuit een Fews rest service</div>

In [1]:
# importeer de benodigde bibliotheken
import os
from pathlib import Path

# voor het plotten van de data
import matplotlib.pyplot as plt
import numpy as np

# Vanuit toolbox gebruiken we standaard 'Config' en 'DataAdapter'.
# De functie die we willen gebruiken om data op te halen is 'LoadsFews'
from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter
from toolbox_continu_inzicht.loads import LoadsFews
from toolbox_continu_inzicht import datetime_from_epoch

# Hiermee kan je markdown printen tijdens uitvoeren van functie
from IPython.display import display, Markdown

**loads_fews_config.yaml:**
```Yaml
GlobalVariables:
    rootdir: "data_sets"   

    csv:
        delimiter: ","

    LoadsFews:
        host: "https:**********"
        port: 8443
        region: "fewspiservice"
        version: "1.25"
        filter: "HKV_WV_1"
        parameters: ["WNSHDB1"]  
        MISSING_VALUE: -9999.0
        moments: [-24,0,24,48]

DataAdapter: 
    locaties: 
        type: csv
        path: "locations.csv"
    waterstanden: 
        type: csv
        path: "waterstanden.csv"
```

**locations.csv:**
```csv
id,name
1,VOV9345
2,VOV32379
```

In [ ]:
# Path naar config bestand
config_file = Path("data_sets/loads_fews_config.yaml")
if os.path.exists(config_file):
    # Maak config aan, deze gebruiken we om de adapter aan te maken
    display(
        Markdown("*Maak config aan, deze gebruiken we om de adapter aan te maken.*")
    )
    config = Config(config_path=config_file)
    config.lees_config()

    display(Markdown("*Maak DataAdapter aan, deze gebruiken bij de functie.*"))
    data_adapter = DataAdapter(config=config)
else:
    display(Markdown(f"**⚠️ Config file '{config_file}' not found.**"))

In [ ]:
# Oude gegevens verwijderen
display(Markdown("Oude gegevens verwijderen"))

output_info = config.data_adapters
output_file = Path(
    config.global_variables["rootdir"] / Path(output_info["waterstanden"]["path"])
)

print(output_file)

if os.path.exists(output_file):
    os.remove(output_file)

In [ ]:
# Gebruik de 'LoadsFews' om gegevens uit de rest-service van Fews te halen
display(
    Markdown("Gebruik de 'LoadsFews' om gegevens uit de rest-service van Fews te halen")
)
fews = LoadsFews(data_adapter=data_adapter, input="locaties", output="waterstanden")

# Omdat de data via een asynchrone functie opgehaald
# wordt is de run ook asynchroon gemaakt
display(Markdown("Voer de functie uit."))
df_out = await fews.run()

In [ ]:
# test of we data hebben opgehaald
if os.path.exists(output_file):
    display(Markdown(f"*{output_file} is aangemaakt.*"))
else:
    display(Markdown(f"*{output_file} is niet aangemaakt.*"))

# controleer de opgehaalde waarden
display(Markdown("**Controleer de opgehaalde waarden**"))

In [31]:
# split de dataframe in tweeën & process
values_nan = df_out[df_out["value"] == 999999999.0].index
for val in values_nan:
    df_out.loc[val, "value"] = np.nan
df_out.sort_index()

df_plot = df_out.set_index(df_out["datetime"].apply(lambda x: datetime_from_epoch(x)))
df_plot = df_plot[df_plot["objectid"] == 1]

measurement_index = df_plot["parameterid"] == 2
df_plot_measurements = df_plot[measurement_index][["value"]]
df_plot_forecast = df_plot[~measurement_index][["value"]]

In [ ]:
fig, ax = plt.subplots()
df_plot_measurements.plot(color="C0", ax=ax)
df_plot_forecast.plot(color="C1", ax=ax)
ax.legend(["Prediction", "Measurement"])